In [82]:
#as always, we start by grabbing our data
import pandas as pd

local ='../data/nyc/nyc-rent-2016.csv'

df = pd.read_csv(local)
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,...,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,...,0,0,0,0,0,0,0,0,0,0


In [83]:
#next we'll split our data into a train and a test set
df['created'] = pd.to_datetime(df['created'], infer_datetime_format = True)
df['month'] = df['created'].dt.month

In [84]:
train = df[df['month']<6]
test = df[df['month']==6]

In [85]:
#confirming number of entries
train.shape, test.shape

((31515, 35), (16785, 35))

In [86]:
#next we'll import our regression model with no hyper paremters changed as a baseline
from sklearn.linear_model import LinearRegression

linr = LinearRegression()

In [87]:
#after looking at these, most are true false, and interest level is the only real candidate for encoding in my opinion
train.columns

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'exclusive', 'terrace', 'loft',
       'garden_patio', 'common_outdoor_space', 'wheelchair_access', 'month'],
      dtype='object')

In [88]:
train['new_construction'].describe()

count    31515.00000
mean         0.05004
std          0.21803
min          0.00000
25%          0.00000
50%          0.00000
75%          0.00000
max          1.00000
Name: new_construction, dtype: float64

In [89]:
#making our X and y test for the regression algorithm
features = ['bedrooms', 'interest_level']
target = 'price'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [90]:
#importing the encoder and running the encoding
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

encoder = ce.OneHotEncoder(use_cat_names = True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [43]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [91]:
linr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [92]:
from sklearn.metrics import mean_absolute_error as mae


y_pred = linr.predict(X_test)

mae(y_test, y_pred)

822.1205242776289

In [96]:
y_pred

array([4188., 4016., 3274., ..., 2706., 4016., 3448.])

In [93]:
#These numbers came out really huge, but the predictions are almost as accurate as last night's so I plan to ask more about this
#later
linr.coef_

array([7.40788024e+02, 1.10768471e+16, 1.10768471e+16, 1.10768471e+16])

In [94]:
linr.intercept_

-1.107684710145826e+16